In [4]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import io 
import os
import pandas as pd
import torchvision
from keras import backend as K
import torch
from torchvision import datasets, models, transforms
import keras
import tensorflow as tf
from keras.applications import vgg16
from keras.models import Sequential,Model
from keras import layers
from tensorflow import keras
from keras.layers import Dense, Conv2D, Conv2DTranspose,MaxPool2D , Flatten, Input
from keras import optimizers
from tensorflow.keras import utils
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, load_model
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization.batch_normalization import BatchNormalization
from imutils import paths
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt

In [3]:
pip install --upgrade imutils

In [33]:
import gc
gc.collect()

# ***Loading Dataset***

In [5]:
#Train path
DATASET_PATH = "../input/isic2017-256x256-jpeg/ISIC_2017_256x256"
# define the path to the images and masks dataset
TRAIN_IMAGE_PATH = os.path.join(DATASET_PATH, "train/images")
TRAIN_MASK_PATH = os.path.join(DATASET_PATH, "train/masks")
print(TRAIN_IMAGE_PATH)
print(TRAIN_MASK_PATH)
train_imagePaths = sorted(list(paths.list_images(TRAIN_IMAGE_PATH)))
train_maskPaths = sorted(list(paths.list_images(TRAIN_MASK_PATH)))

#Test path
TEST_IMAGE_PATH = os.path.join(DATASET_PATH, "test/images")
TEST_MASK_PATH = os.path.join(DATASET_PATH, "test/masks")
print(TEST_IMAGE_PATH)
print(TEST_MASK_PATH)
test_imagePaths = sorted(list(paths.list_images(TEST_IMAGE_PATH)))
test_maskPaths = sorted(list(paths.list_images(TEST_MASK_PATH)))
#Validation path
VALID_IMAGE_PATH = os.path.join(DATASET_PATH, "valid/images")
VALID_MASK_PATH = os.path.join(DATASET_PATH, "valid/masks")
print(VALID_IMAGE_PATH)
print(VALID_MASK_PATH)
valid_imagePaths = sorted(list(paths.list_images(VALID_IMAGE_PATH)))
valid_maskPaths = sorted(list(paths.list_images(VALID_MASK_PATH)))

In [6]:
x_train = []
y_train = []

x_test = []
y_test = []

x_valid = []
y_valid = []
for i,j in zip(train_imagePaths,train_maskPaths):
    image = cv2.imread(i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(j)
    #mask = mask.astype(np.uint8)
    x_train.append(image)
    y_train.append(mask)
    

for i,j in zip(test_imagePaths,test_maskPaths):
    image = cv2.imread(i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(j)
    #mask = mask.astype(np.uint8)
    x_test.append(image)
    y_test.append(mask)
   

for i,j in zip(valid_imagePaths,valid_maskPaths):
    image = cv2.imread(i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(j)
    #mask = mask.astype(np.uint8)
    x_valid.append(image)
    y_valid.append(mask)
    


In [7]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

In [8]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_valid.shape)
print(y_valid.shape)

# ***Plotting Some Samples***

In [ ]:
plt.figure(figsize=(20,9))
plt.subplot(2,4,1)
plt.imshow(x_train[0])
plt.subplot(2,4,2)
plt.imshow(x_train[55])
plt.subplot(2,4,3)
plt.imshow(x_train[130])
plt.subplot(2,4,4)
plt.imshow(x_train[255])
plt.subplot(2,4,5)
plt.imshow(x_train[512])
plt.subplot(2,4,6)
plt.imshow(x_train[1024])
plt.subplot(2,4,7)
plt.imshow(x_train[1350])
plt.subplot(2,4,8)
plt.imshow(x_train[1890])
plt.show()

In [ ]:
plt.figure(figsize=(20,9))
plt.subplot(2,4,1)
plt.imshow(y_train[0], cmap = plt.cm.binary_r)
plt.subplot(2,4,2)
plt.imshow(y_train[55], cmap = plt.cm.binary_r)
plt.subplot(2,4,3)
plt.imshow(y_train[130], cmap = plt.cm.binary_r)
plt.subplot(2,4,4)
plt.imshow(y_train[255], cmap = plt.cm.binary_r)
plt.subplot(2,4,5)
plt.imshow(y_train[512], cmap = plt.cm.binary_r)
plt.subplot(2,4,6)
plt.imshow(y_train[1024], cmap = plt.cm.binary_r)
plt.subplot(2,4,7)
plt.imshow(y_train[1350], cmap = plt.cm.binary_r)
plt.subplot(2,4,8)
plt.imshow(y_train[1890], cmap = plt.cm.binary_r)
plt.show()

# ***Loading VGG16***

In [9]:
#Loading VGG network
#Initializing weights to imagenet 
img_input = Input(shape=(256, 256, 3))
vgg = vgg16.VGG16(include_top=False,weights = 'imagenet', input_tensor=img_input)
for layer in vgg.layers:
   layer.trainable = False
vgg.summary()

# ***Loss and Metric***

In [71]:
from keras import backend as K
def IoU(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.square(y_true), axis = -1) + K.sum(K.square(y_pred), axis=-1)
    ans = (intersection + smooth) / K.abs(sum_ - intersection + smooth)
    return ans

def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.square(y_true), axis = -1) + K.sum(K.square(y_pred), axis=-1)
    ans = (intersection + smooth) / K.abs(sum_ - intersection + smooth)
    return 1 - ans

def IoU_score(y_true, y_pred,smooth = 100):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred) - intersection
    ans = (intersection + smooth) / K.abs(sum_ - intersection + smooth)
    return ans

# ***Trying Different LR***

In [10]:
#Learning rate 1/3
model = Sequential()
model.add(vgg)
model.add(Conv2D(filters=512, kernel_size=(1,1),padding = 'same',activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1),padding = 'same',activation="relu"))
model.add(Conv2D(filters=1, kernel_size=(1,1),padding = 'same',activation="relu"))
model.add(Conv2DTranspose(filters=1 ,padding='same', kernel_size=(64,64),strides=(32,32) ,activation="sigmoid"))
model.summary()
model.compile(loss=[jaccard_distance],
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                   metrics=[IoU])


In [11]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5,restore_best_weights = True)
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1,save_best_only=True,save_freq=5 * 128,mode='auto', period=1)
history = model.fit(x_train,y_train.astype(np.float32),validation_data=(x_valid, y_valid.astype(np.float32)),
                    batch_size=16,
                    epochs=100,callbacks=[es,checkpoint])

In [12]:
from keras import backend as K

#Learning rate 2/3
model = Sequential()
model.add(vgg)
model.add(Conv2D(filters=512, kernel_size=(1,1),padding = 'same',activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1),padding = 'same',activation="relu"))
model.add(Conv2D(filters=1, kernel_size=(1,1),padding = 'same',activation="relu"))
model.add(Conv2DTranspose(filters=1 ,padding='same', kernel_size=(64,64),strides=(32,32) ,activation="sigmoid"))
model.summary()
model.compile(loss=[jaccard_distance],
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                   metrics=[IoU])


In [13]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5,restore_best_weights = True)
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1,save_best_only=True,save_freq=5 * 128,mode='auto', period=1)
history = model.fit(x_train,y_train.astype(np.float32),validation_data=(x_valid, y_valid.astype(np.float32)),
                    batch_size=16,
                    epochs=100,callbacks=[es,checkpoint])

In [90]:
#Learning rate 3/3
model = Sequential()
model.add(vgg)
model.add(Conv2D(filters=512, kernel_size=(1,1),padding = 'same',activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=512, kernel_size=(1,1),padding = 'same',activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=1, kernel_size=(1,1),padding = 'same',activation="relu"))
model.add(BatchNormalization())
model.add(Conv2DTranspose(filters=1 ,padding='same', kernel_size=(64,64),strides=(32,32) ,activation="sigmoid"))
model.summary()
model.compile(loss=[jaccard_distance],
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   metrics=[IoU])


In [91]:
history = model.fit(x_train,y_train.astype(np.float32),validation_data=(x_valid, y_valid.astype(np.float32)),
                    batch_size=16,
                    epochs=100)

# ***Evaluation***

In [92]:
score = model.evaluate(x_test, y_test.astype(np.float32), verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [93]:
def threshold(img):
    sub = (model.predict(img.reshape(1,256,256,3))).flatten()
    for i in range(len(sub)):
        if sub[i] > 0.5:
            sub[i] = 1
        else:
            sub[i] = 0
    return sub

In [94]:
prediction = np.array(model.predict(x_test))
plt.figure(figsize=(20,9))
plt.subplot(2,4,1)
plt.imshow(x_test[470])
plt.title("Original Image")
plt.subplot(2,4,2)
plt.imshow(threshold(x_test[470]).reshape(256,256), plt.cm.binary_r)
plt.title("Predicted Image")
plt.subplot(2,4,3)
plt.imshow(y_test[470])
plt.title("GroundTruth")
plt.subplot(2,4,5)
plt.imshow(x_test[25])
plt.title("Original Image")
plt.subplot(2,4,6)
plt.imshow(threshold(x_test[25]).reshape(256,256), plt.cm.binary_r)
plt.title("Predicted Image")
plt.subplot(2,4,7)
plt.imshow(y_test[25])
plt.title("GroundTruth")
plt.show()
plt.figure(figsize=(20,9))
plt.subplot(2,4,1)
plt.imshow(x_test[35])
plt.title("Original Image")
plt.subplot(2,4,2)
plt.imshow(threshold(x_test[35]).reshape(256,256), plt.cm.binary_r)
plt.title("Predicted Image")
plt.subplot(2,4,3)
plt.imshow(y_test[35])
plt.title("GroundTruth")
plt.figure(figsize=(20,9))
plt.subplot(2,4,5)
plt.imshow(x_test[45])
plt.title("Original Image")
plt.subplot(2,4,6)
plt.imshow(threshold(x_test[45]).reshape(256,256), plt.cm.binary_r)
plt.title("Predicted Image")
plt.subplot(2,4,7)
plt.imshow(y_test[45])
plt.title("GroundTruth")
plt.show()

# ***Success/Fail Cases***

In [95]:
#Bad
iou = IoU_score(prediction[15],y_test[15])
print(iou)
plt.figure(figsize=(20,9))
plt.subplot(1,3,1)
plt.imshow(x_test[15])
plt.title("Original Image")
plt.subplot(1,3,2)
plt.imshow(threshold(x_test[15]).reshape(256,256), plt.cm.binary_r)
plt.title("Predicted Image")
plt.subplot(1,3,3)
plt.imshow(y_test[15])
plt.title("GroundTruth")

In [96]:
#Bad
iou = IoU_score(prediction[45],y_test[45])
print(iou)
plt.figure(figsize=(20,9))
plt.subplot(1,3,1)
plt.imshow(x_test[45])
plt.title("Original Image")
plt.subplot(1,3,2)
plt.imshow(threshold(x_test[45]).reshape(256,256), plt.cm.binary_r)
plt.title("Predicted Image")
plt.subplot(1,3,3)
plt.imshow(y_test[45])
plt.title("GroundTruth")

In [100]:
#Bad
iou = IoU_score(prediction[1],y_test[1])
print(iou)
plt.figure(figsize=(20,9))
plt.subplot(1,3,1)
plt.imshow(x_test[1])
plt.title("Original Image")
plt.subplot(1,3,2)
plt.imshow(threshold(x_test[1]).reshape(256,256), plt.cm.binary_r)
plt.title("Predicted Image")
plt.subplot(1,3,3)
plt.imshow(y_test[1])
plt.title("GroundTruth")

In [97]:
#Good
iou = IoU_score(prediction[470],y_test[470])
print(iou)
plt.figure(figsize=(20,9))
plt.subplot(1,3,1)
plt.imshow(x_test[470])
plt.title("Original Image")
plt.subplot(1,3,2)
plt.imshow(threshold(x_test[470]).reshape(256,256), plt.cm.binary_r)
plt.title("Predicted Image")
plt.subplot(1,3,3)
plt.imshow(y_test[470])
plt.title("GroundTruth")

In [99]:
#Good
iou = IoU_score(prediction[47],y_test[1])
print(iou)
plt.figure(figsize=(20,9))
plt.subplot(1,3,1)
plt.imshow(x_test[47])
plt.title("Original Image")
plt.subplot(1,3,2)
plt.imshow(threshold(x_test[47]).reshape(256,256), plt.cm.binary_r)
plt.title("Predicted Image")
plt.subplot(1,3,3)
plt.imshow(y_test[47])
plt.title("GroundTruth")